In [1]:
from pathlib import Path
import math
import os
import numpy as np
import pandas as pd
import time
from numba import jit

def extract_data():
    raiz = Path.cwd() #capturo diretório atual
    pasta_dados = Path.joinpath(raiz,"dados") #defino onde estão os problemas
    prob_por_arquivo = 10 #informando quantos problemas por arquivo eu tenho
    conjuntos = [10,20,50,100,200,500,1000]

    for conjunto in conjuntos:  #circulo nos arquivos
        dados[conjunto] = {}
        arquivo = f"sch"+str(conjunto)+'.txt'
        print("Iniciando problema {}".format(arquivo))
        caminho_problema = Path.joinpath(pasta_dados,arquivo)
        df = pd.read_fwf(caminho_problema) #importo no df
        df.columns=["n","p","a","b"]
        df.drop(columns=["n"], inplace=True) #jogo fora columa inútil
        n=df.iloc[0,0] #guardo o tamanho de itens do problema
        print("n = ", n)  
        df.dropna(inplace=True) #jogo fora linhas inúteis
        for problema_numero in range(prob_por_arquivo): #vou circular nos problemas dentro do arquivo. Cada iteração deste for é um problema.
            prob = df[n*problema_numero:n*(problema_numero+1)]
            dados[conjunto][problema_numero+1] = {
                'pi': prob["p"].to_numpy(),
                'ai': prob["a"].to_numpy(),
                'bi': prob["b"].to_numpy()
            }
            ## nesse ponto tenho o parâmetros do problema atual guardadas nas variáveis pi, ai e bi, prontos para serem utilizados na modelagem.
    return dados

@jit(nopython=True)
def transforma(solucao):
    set_E = np.nonzero(solucao == True)[0] #[0] para retornar o np array e não o tuple.
    set_T = np.nonzero(solucao == False)[0]
    return set_E,set_T

def transforma_bin(set_E, set_T, conjunto):
    solucao = range(0, conjunto)
    solucao_bin = np.isin(solucao, set_E)
    return solucao_bin

In [2]:
dados = {}
dados = extract_data()

solucoes_pandas = pd.read_pickle('solucoes.pkl')
solucoes = solucoes_pandas.to_dict()


Iniciando problema sch10.txt
n =  10
Iniciando problema sch20.txt
n =  20
Iniciando problema sch50.txt
n =  50
Iniciando problema sch100.txt
n =  100
Iniciando problema sch200.txt
n =  200
Iniciando problema sch500.txt
n =  500
Iniciando problema sch1000.txt
n =  1000


In [3]:
lista_z = [0.25 , 0.5 , 0.6 , 0.75, 2]
conjunto = 10
problema = 1
h = 0.2
for cada_z in lista_z:
    print(cada_z, solucoes[(conjunto,problema,h,cada_z)])

0.25 [False  True False  True False False False False False False]
0.5 [False  True False  True False False False False False False]
0.6 [False  True False  True False False False False False False]
0.75 [False  True False False False False  True False False False]
2 [False  True False False False False  True False False False]


In [87]:
#luiza, vou por as funções aqui, e os testes vou manter nas células abaixo
#aí você consegue ver os testes que eu fiz e como eu espero que a função se comporte.

@jit(nopython=True)
def crossover(p1,p2):     #p1,p2 = np.array com soluções, formato booleano.
    tamanho = len(p1)
    metade = int(tamanho/2)
    parte1 = p1[:metade]
    parte2 = p2[metade:]
    filho = np.append(parte1,parte2)
    return filho #falta incluir uma função de reparo de infactíveis




@jit(nopython=True)
def crossover_r(p1,p2):     #p1,p2 = np.array com soluções, formato booleano.
                            #a diferença desse pro anterior é que dois cruzamentos dos mesmos pais
                            #vão retornar filhos diferentes.
                            #a diferença de tempo para 1000 tarefas é de 2us para 20us.
    roleta =  np.array([np.random.random() for i in range(len(p1))])
    filho = np.copy(p1)
    p2 = np.copy(p2)
    for cada_tarefa in np.where(roleta<=0.5): #o filho é igual p1, exceto onde roleta<=0.5.
        filho[cada_tarefa] = p2[cada_tarefa]
        
    return filho #falta incluir uma função de reparo de infactíveis


@jit(nopython=True)
def mutacao(sol,p): #sol = np.array com solução, formato booleano.
                    #p = probabilidade de cada bit sofrer mutação, float ;  [0,1].
    roleta =  np.array([np.random.random() for i in range(len(sol))])
    copia_sol = np.copy(sol)
    for cada_mutante in np.where(roleta<=p):
        copia_sol[cada_mutante] = np.logical_not(copia_sol[cada_mutante])
    return copia_sol

In [83]:
a=np.tile(True,4)
b=np.tile(False,4)

crossover(a,b)

array([ True,  True, False, False])

In [84]:
a=np.tile(True,1000)
b=np.tile(False,1000)
%timeit crossover(a,b)

2.42 µs ± 398 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [88]:
%timeit crossover_r(a,b)

24.7 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


1

In [80]:
%%timeit
d =  mutacao(a,0.3)

28.7 µs ± 2.66 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [61]:
d

array([False,  True,  True,  True,  True,  True, False,  True, False,
       False, False, False, False,  True, False, False,  True,  True,
       False,  True,  True,  True,  True,  True, False, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True, False,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True,  True,  True,  True, False, False,  True,
        True, False, False, False,  True,  True, False,  True, False,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False, False, False,  True,  True, False,  True, False,  True,
       False,  True,  True, False, False,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
       False,  True,  True, False,  True,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,

In [79]:
a=np.tile(True,1000)
print(sum(a))
d =  mutacao(a,0.3)
print(sum(a))
print(sum(d))

1000
1000
693


506

In [53]:
a

array([ True, False, False,  True,  True, False,  True, False, False,
        True,  True, False, False,  True, False,  True,  True,  True,
       False,  True, False, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True,  True,  True, False,
       False, False,  True,  True, False, False,  True, False,  True,
       False,  True, False, False, False,  True,  True, False, False,
       False, False,  True,  True,  True, False, False, False, False,
        True,  True, False,  True, False,  True, False,  True,  True,
       False, False, False, False,  True,  True, False,  True, False,
        True, False, False, False, False,  True,  True,  True,  True,
       False, False, False, False, False,  True, False,  True, False,
        True, False,  True, False,  True,  True, False, False, False,
        True,  True, False,  True,  True,  True,  True, False, False,
       False, False, False,  True, False,  True,  True, False, False,
        True, False,